In [1]:
import numpy as np
from quant_rotor.core.hamiltonian import hamiltonian
from quant_rotor.core.hamiltonian_big import hamiltonian_big
from itertools import product
from quant_rotor.core.t_amplitudes_non_periodic import t_non_periodic
from quant_rotor.core.t_amplitudes_periodic import t_periodic
from quant_rotor.models.density_matrix import density_matrix_1
from scipy.sparse import diags 
from quant_rotor.models.t_amplitudes_guess import intermediate_normalisation, t_1_amplitude_guess_ground_state, t_2_amplitude_guess_ground_state, amplitute_energy

# t_1 amplitutdes. (work)

In [ ]:
sites = 3
states = 5

g = 1
l_val = 0

H = hamiltonian(states, sites, g, l_val)[0]

eig_val, eig_vec = np.linalg.eig(H)

In [ ]:
num_sites=5
num_states=5

In [318]:
def generate_quantum_rotor_configurations(num_sites: int, num_states: int) -> np.ndarray:
    """
    Generate all possible configurations for a quantum rotor system.
    
    Args:
        num_sites (int): Number of rotor sites.
        num_states (int): Number of discrete states per rotor.
                          Typically, this is 2*m_max + 1 for m in [-m_max, ..., m_max].
    
    Returns:
        np.ndarray: Array of shape (num_states ** num_sites, num_sites) containing all configurations.
    """
    return np.array(list(product(range(num_states), repeat=num_sites)))

# Example usage:
configs = generate_quantum_rotor_configurations(num_sites, num_states)
# print(configs)

In [319]:
for site in range(num_sites):
    for state in range(1,num_states):

        print("site:", site, "state:", state)

        index = state*num_states**site
        print("state", configs[index])

site: 0 state: 1
state [0 0 0 0 1]
site: 0 state: 2
state [0 0 0 0 2]
site: 0 state: 3
state [0 0 0 0 3]
site: 0 state: 4
state [0 0 0 0 4]
site: 1 state: 1
state [0 0 0 1 0]
site: 1 state: 2
state [0 0 0 2 0]
site: 1 state: 3
state [0 0 0 3 0]
site: 1 state: 4
state [0 0 0 4 0]
site: 2 state: 1
state [0 0 1 0 0]
site: 2 state: 2
state [0 0 2 0 0]
site: 2 state: 3
state [0 0 3 0 0]
site: 2 state: 4
state [0 0 4 0 0]
site: 3 state: 1
state [0 1 0 0 0]
site: 3 state: 2
state [0 2 0 0 0]
site: 3 state: 3
state [0 3 0 0 0]
site: 3 state: 4
state [0 4 0 0 0]
site: 4 state: 1
state [1 0 0 0 0]
site: 4 state: 2
state [2 0 0 0 0]
site: 4 state: 3
state [3 0 0 0 0]
site: 4 state: 4
state [4 0 0 0 0]


In [320]:
for i in range(num_sites):
    for j in range(1,num_states):
        for x in range(num_sites):
            for y in range(1,num_states):

                if i < x:
                    index = j*num_states**i + y*num_states**x
                    print("state", configs[index])

state [0 0 0 1 1]
state [0 0 0 2 1]
state [0 0 0 3 1]
state [0 0 0 4 1]
state [0 0 1 0 1]
state [0 0 2 0 1]
state [0 0 3 0 1]
state [0 0 4 0 1]
state [0 1 0 0 1]
state [0 2 0 0 1]
state [0 3 0 0 1]
state [0 4 0 0 1]
state [1 0 0 0 1]
state [2 0 0 0 1]
state [3 0 0 0 1]
state [4 0 0 0 1]
state [0 0 0 1 2]
state [0 0 0 2 2]
state [0 0 0 3 2]
state [0 0 0 4 2]
state [0 0 1 0 2]
state [0 0 2 0 2]
state [0 0 3 0 2]
state [0 0 4 0 2]
state [0 1 0 0 2]
state [0 2 0 0 2]
state [0 3 0 0 2]
state [0 4 0 0 2]
state [1 0 0 0 2]
state [2 0 0 0 2]
state [3 0 0 0 2]
state [4 0 0 0 2]
state [0 0 0 1 3]
state [0 0 0 2 3]
state [0 0 0 3 3]
state [0 0 0 4 3]
state [0 0 1 0 3]
state [0 0 2 0 3]
state [0 0 3 0 3]
state [0 0 4 0 3]
state [0 1 0 0 3]
state [0 2 0 0 3]
state [0 3 0 0 3]
state [0 4 0 0 3]
state [1 0 0 0 3]
state [2 0 0 0 3]
state [3 0 0 0 3]
state [4 0 0 0 3]
state [0 0 0 1 4]
state [0 0 0 2 4]
state [0 0 0 3 4]
state [0 0 0 4 4]
state [0 0 1 0 4]
state [0 0 2 0 4]
state [0 0 3 0 4]
state [0 0

In [275]:
def intermediate_normalisation(eig_val: np.ndarray, eig_vec: np.ndarray)->np.ndarray:

    min_index = np.argmin(eig_val)

    reference_ground_state = eig_vec[:, min_index]

    d = reference_ground_state / reference_ground_state[0]

    return d

def t_1_amplitutde(site_a: int, state_a: int, states: int, d: np.ndarray)-> int:
    index = state_a*states**site_a
    return d[index]

def t_2_amplitutde(site_a: int, state_a: int, site_b: int, state_b: int, states: int, d: np.ndarray)-> int:
    t_1_a = t_1_amplitutde(site_a, state_a, states, d)
    t_1_b = t_1_amplitutde(site_b, state_b, states, d)

    C_ab = d[state_a*states**site_a + state_b*states**site_b]

    t_2_ab = C_ab - t_1_a*t_1_b

    return t_2_ab, C_ab

In [304]:
d = intermediate_normalisation(eig_val, eig_vec)
site_a = 0
state_a = 1
site_b = 1
state_b = 1
t_1_a = t_1_amplitutde(site_b, state_b, states, d)
t_2_ab = t_2_amplitutde(site_a, state_a, site_b, state_b, states, d)
print(t_1_a)
print(t_2_ab[0])

(-5.425056681426581e-18+0j)
(-0.0394014771952756+0j)


In [277]:
site_c = 2
state_c = 1
site_d = 0
state_d = 1
t_1_c = t_1_amplitutde(site_c, state_c, states, d)
t_2_cd = t_2_amplitutde(site_c, state_c, site_d, state_d, states, d)

print(t_1_c)
print(t_2_cd[0])

(-1.6540065711986537e-16+0j)
(-0.03926454595099672+0j)


# t_amplitudes. (final)

In [16]:
# def intermediate_normalisation(eig_val: np.ndarray, eig_vec: np.ndarray)->np.ndarray:

#     min_index = np.argmin(eig_val)

#     reference_ground_state = eig_vec[:, min_index]

#     d = reference_ground_state / reference_ground_state[0]

#     return d

# def t_1_amplitutde(site_a: int, state_a: int, states: int, d: np.ndarray)-> int:
#     index = state_a*states**site_a
#     return d[index]

# def t_2_amplitutde(site_a: int, state_a: int, site_b: int, state_b: int, states: int, d: np.ndarray)-> int:
#     t_1_a = t_1_amplitutde(site_a, state_a, states, d)
#     t_1_b = t_1_amplitutde(site_b, state_b, states, d)

#     C_ab = d[state_a*states**site_a + state_b*states**site_b]

#     t_2_ab = C_ab - t_1_a*t_1_b

#     return t_2_ab, C_ab

# t_amplitudes energy. (work)

In [327]:
def amplitute_energy(sites: int, states: int, g: float, d: np.ndarray):

    K, V = h.write_matrix_elements((states-1) // 2)

    V = V + V.T - np.diag(np.diag(V))
    V = V.reshape(states,states,states,states)
    V *= g 

    K = h.basis_m_to_p_matrix_conversion(K)
    V = h.basis_m_to_p_matrix_conversion(V)

    E_0 = K[0,0]*sites + np.einsum("ijij->", V) * states**sites

    sum_t_1 = 0
    sum_t_2 = 0

    for site_a in range(sites):
        for state_a in range(1, states):

            t1 = t_1_amplitutde(site_a, state_a, states, d)
            sum_t_1 += K[state_a, 0] * t1

            for site_b in range(site_a + 1, sites):
                for state_b in range(1, states):
                    
                    C2 = t_2_amplitutde(site_a, state_a, site_b, state_b, states, d)[1]
                    sum_t_2 += V[state_a, state_b, 0, 0] * C2

    print(E_0)
    print(sum_t_1)
    print(sum_t_2)

    return E_0 + sum_t_1 + sum_t_2
        

In [328]:
amplitute_energy(sites, states, g, d)

0.0
0j
(-0.020101729050886804+0j)


(-0.020101729050886804+0j)

In [326]:
min_index = np.argmin(eig_val)
eig_val[min_index]

(-0.02010172905076164+0j)

# t_amplitudes energy. (final)

In [16]:
def amplitute_energy(sites: int, states: int, g: float, d: np.ndarray):

    K, V = h.write_matrix_elements((states-1) // 2)

    V = V + V.T - np.diag(np.diag(V))
    V = V.reshape(states,states,states,states)
    V *= g 

    K = h.basis_m_to_p_matrix_conversion(K)
    V = h.basis_m_to_p_matrix_conversion(V)

    E_0 = K[0,0]*sites + np.einsum("ijij->", V) * states**sites

    sum_t_1 = 0
    sum_t_2 = 0

    for site_a in range(sites):
        for state_a in range(1, states):

            t1 = t_1_amplitutde(site_a, state_a, states, d)
            sum_t_1 += K[state_a, 0] * t1

            for site_b in range(site_a + 1, sites):
                for state_b in range(1, states):
                    
                    C2 = t_2_amplitutde(site_a, state_a, site_b, state_b, states, d)[1]
                    sum_t_2 += V[state_a, state_b, 0, 0] * C2

    return E_0 + sum_t_1 + sum_t_2
        

# t_amplitudes Bryan. (work)

In [ ]:
sites = 3
states = 5

g = 1
l_val = 0

H = hamiltonian(states, sites, g, l_val)[0]

eig_val, eig_vec = np.linalg.eig(H)

In [ ]:
def t_1_amplitude_guess_ground_state(states: int, sites: int, g: float, low_states: int, eig_vec: np.ndarray, eig_val: np.ndarray):

    i = low_states
    a = states - low_states
    t_a_i_tensor = np.full((sites, a, i), 0, dtype=complex)

    d = intermediate_normalisation(eig_val, eig_vec)

    for site in range(sites):
        for state in range(states - 1):
            t_a_i_tensor[site, state, i-1] = t_1_amplitutde(site, state + 1, states, d)

    return t_a_i_tensor

def t_2_amplitude_guess_ground_state(states: int, sites: int, g: float, low_states: int, eig_vec: np.ndarray, eig_val: np.ndarray):

    i = low_states
    a = states - low_states
    t_ab_ij_tensor = np.full((sites, sites, a, a, i, i), 0, dtype=complex)

    d = intermediate_normalisation(eig_val, eig_vec)

    for site_a in range(sites):
        for state_a in range(a):
            for site_b in range(site_a + 1, sites):
                for state_b in range(a):

                    t_2_guess = t_2_amplitutde(site_a, state_a + 1, site_b, state_b + 1, states, d)[0]


                    t_ab_ij_tensor[site_a, site_b, state_a, state_b, 0, 0] = t_2_guess
                    t_ab_ij_tensor[site_b, site_a, state_b, state_a, 0, 0] = t_2_guess

    return t_ab_ij_tensor

In [32]:
t_a_i_tensor_guess = t_1_amplitude_guess_ground_state(states, sites, g, 1, eig_vec, eig_val)
t_ab_ij_tensor_guess = t_2_amplitude_guess_ground_state(states, sites, g, 1, eig_vec, eig_val)

d = intermediate_normalisation(eig_val, eig_vec)
energy_guess = amplitute_energy(sites, states, g, d)

print("Energy:", energy_guess)
print(f"1 max: {np.max(np.abs(t_a_i_tensor_guess))}")
print(f"2 max: {np.max(np.abs(t_ab_ij_tensor_guess))}\n")

energy_periodic, t_a_i_tensor_periodic, t_ab_ij_tensor_periodic = t_periodic(sites, states, 1, t_a_i_tensor_guess, t_ab_ij_tensor_guess, 1e-8, g, 3,  False, 3, False, "sin")

print("Energy:", energy_periodic)
print(f"1 max: {np.max(np.abs(t_a_i_tensor_periodic))}")
print(f"2 max: {np.max(np.abs(t_ab_ij_tensor_periodic))}\n")

energy_non_periodic, t_a_i_tensor_non_periodic, t_ab_ij_tensor_non_periodic = t_non_periodic(sites, states, 1, t_a_i_tensor_guess, t_ab_ij_tensor_guess, 1e-8, g, 3, False, 3, False)

print("Energy:", energy_non_periodic)
print(f"1 max: {np.max(np.abs(t_a_i_tensor_non_periodic))}")
print(f"2 max: {np.max(np.abs(t_ab_ij_tensor_non_periodic))}/n")

Energy: (-0.020101729050886804+0j)
1 max: 0.0011068208301983436
2 max: 0.03940147719527568

Energy: (-0.020275691354355526+0j)
1 max: 0.0010918771775619148
2 max: 0.039735692511960824

Energy: (-0.011613363849927137+0j)
1 max: 0.0010141204033551929
2 max: 0.039735692511960824/n


# K and L.

In [67]:
L_sparce = diags([1/np.sqrt(2), 0, 1/np.sqrt(2)], offsets=[-1, 0, 1], shape=(states, states))

L_dense = L_sparce.toarray()

In [68]:
L_dense

array([[0.            , 0.707106781187, 0.            , 0.            , 0.            ],
       [0.707106781187, 0.            , 0.707106781187, 0.            , 0.            ],
       [0.            , 0.707106781187, 0.            , 0.707106781187, 0.            ],
       [0.            , 0.            , 0.707106781187, 0.            , 0.707106781187],
       [0.            , 0.            , 0.            , 0.707106781187, 0.            ]])

# Analisis. (work)

In [16]:
sites = 3
states = 11

g = 5
l_val = 0

H = hamiltonian(states, sites, g, l_val)[0]

eig_val, eig_vec = np.linalg.eig(H)

In [2]:
def convert_i_to_site_state(states: int, sites: int, looking_for_site: int) -> np.ndarray:

    for x in range(sites):

        n_lambda = states**(x)
        n_mu = states**(sites - x - 1)
        # Loop over the local states at the chosen site
        for p in range(states):

            # Sum over all other sites (Lambda and mu represent other sites)
            for Lambda in range(int(n_lambda)):
                for mu in range(int(n_mu)):

                    # Full indices in the Hilbert space
                    i = mu + p * n_mu + Lambda * states * n_mu

                    if i == looking_for_site:
                        return x, p

In [ ]:
index_array = np.argsort(eig_val)

index_1 = index_array[0]
index_2 = index_array[1]

print(index_1)

site, state = convert_i_to_site_state(states, sites, index_1)

print("State:", state, "Site:", site, "\n")

D_1 = density_matrix_1(states, sites, eig_vec[index_1], site)

print("Dencity One site:\n", D_1, "\n")
print("Ground Energystate:\n", eig_val[index_1], "\n")

eigenvector_1 = eig_vec[:, index_1]
eigenvector_2 = eig_vec[:, index_2]

print("Eigenstate:\n", eigenvector_1, "\n")
print("Eigenstate:\n", eigenvector_2, "\n")

0
State: 0 Site: 0 

Dencity One site:
 [[ 0.42595549513 +0.j  0.000000002215+0.j  0.            +0.j  0.            +0.j  0.000003785159+0.j -0.000000000075+0.j -0.000169984369+0.j -0.000000000632+0.j -0.            +0.j -0.000071006378+0.j -0.            +0.j]
 [ 0.000000002215+0.j  0.271683104324+0.j  0.            +0.j -0.000000000045+0.j  0.            +0.j  0.000000000868+0.j  0.000069861538+0.j -0.            +0.j  0.000000001984+0.j -0.            +0.j  0.            +0.j]
 [ 0.            +0.j  0.            +0.j  0.142621489893+0.j  0.            +0.j -0.00000011743 +0.j  0.            +0.j -0.            +0.j  0.000007920261+0.j -0.            +0.j  0.000000000099+0.j  0.            +0.j]
 [ 0.            +0.j -0.000000000045+0.j  0.            +0.j  0.092242108618+0.j  0.            +0.j -0.            +0.j  0.            +0.j  0.            +0.j  0.000307102801+0.j  0.            +0.j -0.            +0.j]
 [ 0.000003785159+0.j  0.            +0.j -0.00000011743 +0.j  0.   

In [20]:
t_1 = t_1_amplitude_guess_ground_state(states, sites, g, 1, eig_vec, eig_val)
# t_1_g = t_1_amplitutde(0, 1, states, d)
t_2 = t_2_amplitude_guess_ground_state(states, sites, g, 1, eig_vec, eig_val)

# print("T_1:\n", t_1, "\n")
# # print("T_1_g:\n", t_1_g, "\n")
# print("T_2:\n", t_2[0], "\n")
# print("C_2:\n", t_2[1], "\n")

In [9]:
def hamiltonian_big_1(state: int, site: int, g_val: float, l_val: float, H_K_V: list[np.ndarray]) -> np.ndarray:

    print("Tests")

    H = H_K_V[0]
    K = H_K_V[1]
    V = H_K_V[2]

    state_old = int(K.shape[0])
    site_old = int(np.log(H.shape[0]) / np.log(state_old))

    eig_val, eig_vec = np.linalg.eig(H)

    index = np.argsort(eig_val)
    psi_vec = eig_vec[:, index[0]] 
    rho_site_0 = density_matrix_1(state_old, site_old, psi_vec, 0)
    eig_val_D, eig_vec_D = np.linalg.eig(rho_site_0)

    matrix_p_to_natural_orbital = eig_vec_D[:, 0:state]

    print("Orbitals:\n", matrix_p_to_natural_orbital)

    # print("System sites:", site)
    # print("Shape p_to_mu:", matrix_p_to_natural_orbital.shape)
    # print("Shape K:", K.shape)
    # print("Shape V:", V.shape)
    # print("Shape H:", H.shape)

    V = V.reshape(state_old**2,state_old**2)
    K_mu = matrix_p_to_natural_orbital.T.conj() @ K @ matrix_p_to_natural_orbital
    V_mu = np.kron(matrix_p_to_natural_orbital.T.conj(), matrix_p_to_natural_orbital.T.conj()) @ V @ np.kron(matrix_p_to_natural_orbital, matrix_p_to_natural_orbital)
    V_mu = V_mu.reshape(state,state,state,state)

    H_mu = hamiltonian(state, site, g_val, l_val, K_mu, V_mu, True)[0]

    return H_mu, K_mu, V_mu, matrix_p_to_natural_orbital

In [ ]:
sites = 3
states = 11

g = 1
l = 0

H_K_V = hamiltonian(11, 3, g, l)

print(H_K_V[0].shape)

print("3 done")

for curent_site in range(3, 3, 2):

    H_K_V = hamiltonian_big_1(states, curent_site, g, l , H_K_V)
    index_array = np.argsort(eig_val)

eig_val, eig_vec = np.linalg.eig(H_K_V[0])

(1331, 1331)
3 done
Tests
Orbitals:
 [[ 0.977125680405+0.j  0.            +0.j  0.212643333721-0.j -0.            +0.j -0.            +0.j  0.002866586357+0.j  0.            +0.j  0.            +0.j  0.            +0.j -0.            -0.j -0.            -0.j]
 [-0.            +0.j  0.706594933912+0.j  0.            +0.j  0.704075332706+0.j  0.            -0.j  0.            +0.j  0.026899273299+0.j  0.065404528195+0.j -0.            +0.j  0.000416613166+0.j  0.000168718063+0.j]
 [ 0.            -0.j -0.706594933912+0.j  0.            -0.j  0.704075332706-0.j  0.            -0.j  0.            +0.j -0.026899273299+0.j  0.065404528195+0.j -0.            +0.j  0.000416613166+0.j -0.000168718063+0.j]
 [-0.15036457953 -0.j  0.            +0.j  0.690707297033+0.j -0.            -0.j -0.706918690438-0.j  0.017717873707+0.j  0.            -0.j  0.            +0.j -0.016308436763+0.j -0.            -0.j -0.            +0.j]
 [-0.15036457953 -0.j -0.            +0.j  0.690707297033-0.j -0.      

In [4]:
index_array = np.argsort(eig_val)

index_1 = index_array[0]
index_2 = index_array[1]

print(index_1)

site, state = convert_i_to_site_state(states, sites, index_1)

print("State:", state, "Site:", site, "\n")

D_1 = density_matrix_1(states, sites, eig_vec[index_1], site)

print("Dencity One site:\n", D_1, "\n")
print("Ground Energystate:\n", eig_val[index_1], "\n")

eigenvector_1 = eig_vec[:, index_1]
eigenvector_2 = eig_vec[:, index_2]

print("Eigenstate:\n", eigenvector_1, "\n")
print("Eigenstate:\n", eigenvector_2, "\n")

t_1 = t_1_amplitude_guess_ground_state(states, sites, g, 1, eig_vec, eig_val)
# t_1_g = t_1_amplitutde(0, 1, states, d)
t_2 = t_2_amplitude_guess_ground_state(states, sites, g, 1, eig_vec, eig_val)

# print("T_1:\n", t_1, "\n")
# print("T_2:\n", t_2, "\n")

10
State: 0 Site: 0 

Dencity One site:
 [[ 0.000001837553+0.j              0.000000005578+0.000000000001j -0.00000274478 +0.000000000127j  0.000000961014-0.000000000387j  0.000000010573-0.000000000207j  0.000000734221+0.000000025803j  0.000000645725-0.000000052325j  0.000003339537-0.000000003231j -0.000000140045+0.000000000002j -0.000006148787-0.00000039611j  -0.000031344181+0.000000000086j]
 [ 0.000000005578-0.000000000001j  0.000008841951+0.j              0.000000500513-0.000000002588j -0.000007233797-0.000000046916j -0.000002498562+0.00000000011j   0.000006796156+0.000000103369j -0.000004965782-0.000000016181j -0.000013940652-0.000000000564j  0.000002590939+0.000000045229j  0.000027049829+0.000000017049j  0.000010333244+0.000000037617j]
 [-0.00000274478 -0.000000000127j  0.000000500513+0.000000002588j  0.000109445365+0.j              0.000015484768-0.000000391595j  0.000011379677-0.000000000254j -0.000001075707-0.000000003627j  0.000003747624-0.000000065589j -0.000010978777-0.00000

In [ ]:
d = intermediate_normalisation(eig_val, eig_vec)
energy_guess = amplitute_energy(sites, states, g, d)

print("Energy:", energy_guess)
print(f"1 max: {np.max(np.abs(t_1))}")
print(f"2 max: {np.max(np.abs(t_2))}\n")

energy_periodic, t_a_i_tensor_periodic, t_ab_ij_tensor_periodic = t_periodic(sites, states, 1, t_1, t_2, 1e-8, g, 3,  False, 3, True, "sin")

print("Energy:", energy_periodic)
print(f"1 max: {np.max(np.abs(t_a_i_tensor_periodic))}")
print(f"2 max: {np.max(np.abs(t_ab_ij_tensor_periodic))}\n")

energy_non_periodic, t_a_i_tensor_non_periodic, t_ab_ij_tensor_non_periodic = t_non_periodic(sites, states, 1, t_1, t_2, 1e-8, g, 3, False, 3, False)

print("Energy:", energy_non_periodic)
print(f"1 max: {np.max(np.abs(t_a_i_tensor_non_periodic))}")
print(f"2 max: {np.max(np.abs(t_ab_ij_tensor_non_periodic))}/n")

Energy: (-2.2484545863483265-1.5945341387334945e-30j)
1 max: 0.132174132203388
2 max: 0.396679753380345

1 max: (-0.036116360660773617+8.41543102654872e-31j)
2 max: (0.8169585430088914-1.0847209346310788e-30j)
Iteration #: 1
Energy: -4.590885037517814

1 max: (0.502227352818398-8.030453615090813e-31j)
2 max: (-0.8477288474302642-5.688655245129979e-31j)
Iteration #: 2
Energy: -7.010481757965623

1 max: (0.06402933371514441+8.932039828055811e-31j)
2 max: (-1.3865044997335656+4.831180993078917e-31j)
Iteration #: 3
Energy: -11.080178120830439

1 max: (0.5943331042161089-1.9053821444339e-30j)
2 max: (1.7905358754546614+7.600358362285579e-31j)
Iteration #: 4
Energy: -16.442175622479926

1 max: (-0.7975619536571539+4.612048359582522e-30j)
2 max: (-3.30093581411503+9.580597196311572e-31j)
Iteration #: 5
Energy: -26.205397984835358

1 max: (4.1078428921210435-1.7975581142723118e-29j)
2 max: (2.699731342169063+7.536996544980011e-30j)
Iteration #: 6
Energy: -34.18962624955452

1 max: (-22.0146688

ValueError: Diverges